# Bayes by Backprop: Mushroom Contextual Bandit

__Background - Multi-armed and Contextual Bandit problems:__ <br>
_“Bandit”_ in _“multi-armed bandits”_ comes from _“one-armed bandit”_ machines used in a casino. Imagine that you are in a casino with many one-armed bandit machines. Each machine has a different probability of a win. Your goal is to maximize total payout. You can pull a limited number of arms, and you don’t know which bandit to use to get the best payout. The problem involves exploration/exploitation tradeoff: you should balance between trying different bandits to learn more about an expected payout of every machine, but you also want to exploit the best bandit you know about more. While multi-armed bandits select the strategy which maximises the expected reward without taking into consideration the state of the environment, _contextual bandits_ output an action based on the context.

__Mushroom Case Stuty:__ <br>
We are provided with a list of 8124 mushrooms, each having 22 features (characteristcs of the mushroom) and 1 label (poisonous or edible). Our agent can carry out 2 actions: eat a mushroom or not eat a mushroom. The problem context is the vector of features which is associated with the mushroom which the agent is about to eat/non eat.
If our agent eats an edible mushroom, it receives a reward of 5. If the agent eats a poisonous mushroom, it receives a reward of -35 with 0.5 probability and a reward of 5 with 0.5 probaility. If the agent doesn't eat, it receives a reward o 0.

We are also provided an _oracle_. The oracle always selects the right action, receiving a reward of 5 when it eats an edible mushroom, and a reward of 0 when it doesn't eat.

__Objective:__ <br>
Create a BNN which minimises the _cumulative regret_ of the agent. Regreat measures the difference between the oracle and our agent's reward.

__BNN Architecture:__
- 2 hidden layers
- Each layer has 100 rectified units
- Vector input: vector consisting of the mushroom features (context) and a one of _K_ encoding of the action
- Output: single scalar representing the expected reward of the given action in the given context

__Additional Important Information:__
- To calculate the expected reward for an action, Google samples twice the weights and averages the outputs
- To train the network, Google keeps a buffer of 4096 mushrooms.
- For training, Google randomly draws minibatches of size 64 for 64 training steps.
- After every training stage, the agent interacts with a new mushroom



In [194]:
# Import Libraries

import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import preprocessing
from sklearn import metrics
import random
import sys
sys.path.append('../../')
from BayesBackpropagation import *
import warnings
warnings.filterwarnings('ignore')

## __Data Preparation:__ ## 
Input is the data file. Output is an array of one-hot encoded labels (y, shape: 8124) and an array of context features (x, shape: 8124 elements, each with 117 features). 117 features derive from doing a one-hot encoding on the feature classes.

In [195]:
# Import data from file
df = pd.read_csv(os.getcwd() + '/agaricus-lepiota.data', sep=',', header=None,
             error_bad_lines=False, warn_bad_lines=True, low_memory=False)

# Set pandas to output all of the columns in output
df.columns = ['class','cap-shape','cap-surface','cap-color','bruises','odor','gill-attachment',
         'gill-spacing','gill-size','gill-color','stalk-shape','stalk-root',
         'stalk-surf-above-ring','stalk-surf-below-ring','stalk-color-above-ring','stalk-color-below-ring',
         'veil-type','veil-color','ring-number','ring-type','spore-color','population','habitat']

# Split context from label
X = pd.DataFrame(df, columns=df.columns[1:len(df.columns)], index=df.index)
# Put the class values (0th column) into Y
Y = df['class']

# Transform labels into one-hot encoded array
le = preprocessing.LabelEncoder()
le.fit(Y)
y = le.transform(Y)

# Temporary variable to avoid error 
x_tmp = pd.DataFrame(X,columns=[X.columns[0]])

# Encode each feature column and add it to x_train 
for colname in X.columns:
    le.fit(X[colname])
    #print(colname, le.classes_)
    x_tmp[colname] = le.transform(X[colname])

# Produce mushroom array: 8124 mushrooms, each with 117 one-hot encoded features
oh = preprocessing.OneHotEncoder(categorical_features='all')
oh.fit(x_tmp)
x = oh.transform(x_tmp).toarray()

## __Reward function:__ ##
Implemented as described above

In [196]:
x.shape

(8124, 117)

In [197]:
# REWARD FUNCTION

def get_reward(eaten, edible):
    # REWARDS FOR AGENT
    #  Eat poisonous mushroom
    if not eaten:
        return 0
    if eaten and edible:
        return 5
    elif eaten and not edible:
        return (5 if np.random.rand() > 0.5 else -35)

def oracle_reward(edible):
    return 5*edible    

## Create Mushroom Buffer
Generate buffer to keep 4096 elements used to train the Bayesian Neural Network

In [198]:
# Initialise the buffer, each element includes an action, context and reward

def init_data():
    contexts, types, optimal_rewards = [], [], []
    for i in np.random.choice(range(len(x)), 10000):
        contexts.append(x[i])
        types.append(y[i])
        optimal_rewards.append(oracle_reward(y[i]))
    return contexts, types, optimal_rewards

contexts, types, optimal_rewards = init_data()


## Neural Network


In [199]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
print("Cuda available?: ",torch.cuda.is_available())

PI = 0.5
SIGMA_1 = torch.FloatTensor([math.exp(-0)])
SIGMA_2 = torch.FloatTensor([math.exp(-6)])

Cuda available?:  False


### Create class for Gaussian distribution

In [200]:
import torch.nn as nn
import torch.nn.functional as F
import sys

Var = lambda x, dtype=torch.FloatTensor: Variable(torch.from_numpy(x).type(dtype))


class MushroomNet():
    def __init__(self, label = 'MushNet', n_weight_sampling=2):
        self.label = label
        self.n_weight_sampling = n_weight_sampling
        self.epsilon = 0
        self.net = None
        self.loss, self.optimizer = None, None
        self.cum_regrets = [0]
        self.bufferX = []
        self.bufferY = []
        self.trainingX = []
        self.trainingY = []
        
    # Use NN to decide next action    
    def try_ (self, mushroom):
        samples = self.n_weight_sampling
        context, edible = contexts[mushroom], types[mushroom]
        try_eat = Var(np.concatenate((context, [1, 0])))
        try_reject = Var(np.concatenate((context, [0, 1])))
        
        # Calculate rewards using model
        with torch.no_grad():
            r_eat = np.mean([self.net.forward(try_eat).numpy() for _ in range(samples)])
            r_reject = np.mean([self.net.forward(try_reject).numpy() for _ in range(samples)])
                         
        # Take random action for epsilon greedy agents, calculate agent's reward
        eaten = r_eat > r_reject
        if np.random.rand()<self.epsilon:
            eaten = (np.random.rand()<.5)
        agent_reward = get_reward(eaten, edible)
        
        # Get rewards and update buffer
        if eaten:
            action = [1, 0]
        else:
            action = [0, 1]
        self.bufferX.append(np.concatenate((context, action)))
        self.bufferY.append(agent_reward)
    
        # Calculate regret
        oracle = oracle_reward(edible)
        regret = oracle - agent_reward
        self.cum_regrets.append(self.cum_regrets[-1]+regret) 
    
        
    # Function for generating a minibatch
    def generate_minibatch(self, trainingX, trainingY):
        bX = []
        bY = []
        for i in range(64):
            random = np.random.randint(0, len(trainingX))
            bX.append(self.trainingX[random])
            bY.append(self.trainingY[random])
            #print("random mushroom is:" + str(random))
            #print("len of bX is:" + str(len(bX)))
        return bX, bY
    
    # Feed next mushroom
    def update(self, mushroom):
        self.try_(mushroom)
        #print("len of buffer is: " + str(len(self.bufferX)))
        self.trainingX = self.bufferX[-4096:]
        #print("len of trainingX is: " + str(len(self.trainingX)))
        self.trainingY = self.bufferY[-4096:]
        for minibatch in range(64):
            bX, bY = self.generate_minibatch(self.trainingX, self.trainingY)
            self.net.zero_grad()
            self.loss(Var(np.asarray(bX)), Var(np.asarray(bY))).backward()
            self.optimizer.step()
        

In [201]:
# Class for BBB agent
class BBB_MNet(MushroomNet):
    def __init__(self, label):
        super().__init__(label)
        self.net = BayesianNetwork(inputSize = x.shape[1]+2,
                          CLASSES = 1, 
                          layers=np.array([100,100]), 
                          activations = np.array(['relu','relu','none']), 
                          SAMPLES = 2, 
                          BATCH_SIZE = 64,
                          NUM_BATCHES = 64,
                          hasScalarMixturePrior=True,
                          PI=PI,
                          SIGMA_1 = SIGMA_1,
                          SIGMA_2 = SIGMA_2
                          ).to(DEVICE)
        self.optimizer = optim.Adam(self.net.parameters(), lr = 0.5)
        self.loss = lambda data, target:self.net.BBB_loss(data, target)
        
        
# Class for Greedy agents
class EpsGreedyMlp(MushroomNet):
    def __init__(self, epsilon=0, **kwargs):
        super().__init__(**kwargs)
        self.n_weight_sampling = 1
        self.epsilon = epsilon
        self.net = nn.Sequential(
        nn.Linear(x.shape[1]+2, 100), nn.ReLU(),
        nn.Linear(100, 100), nn.ReLU(),
        nn.Linear(100, 1))
        self.optimizer = optim.SGD(self.net.parameters(), lr = 0.001)
        self.mse = nn.MSELoss()
        self.loss = lambda data, target: self.mse(self.net.forward(data), target)

In [202]:
mushroom_nets = {'bbb':BBB_MNet(label = 'BBB'),
                 'e0':EpsGreedyMlp(epsilon=0, label = 'Greedy'),
                 'e1':EpsGreedyMlp(epsilon=0.01, label = '1% Greedy'),
                 'e5':EpsGreedyMlp(epsilon=0.05, label = '5% Greedy')}

In [203]:
import plotly.graph_objs as go
class Fig(go.FigureWidget):
    def __init__(self):
        super().__init__([go.Scatter(name = net.label, x=[], y=[]) 
                          for i, net in mushroom_nets.items()])
        self.layout.title = 'Cumulative Regrets'
    
    def update(self, dic):
        for j, (lb, y) in enumerate(dic.items()):
            self.data[j].x = list(range(len(y)))
            self.data[j].y = y
    
    def save(self, file_name = 'mushroom_regrets.csv'):
        dic = {fd.name: fd.y for fd in self.data}
        pd.DataFrame.from_dict(dic).to_csv(file_name)
        
        
fig = Fig()
fig  

Fig({
    'data': [{'name': 'BBB', 'type': 'scatter', 'uid': '61245c8e-c301-4e03-8ed9-bf9a6b945706', 'x': [], 'y': []},
             {'name': 'Greedy', 'type': 'scatter', 'uid': 'ea767dae-7e79-402b-b47f-268bf219f548', 'x': [], 'y': []},
             {'name': '1% Greedy', 'type': 'scatter', 'uid': 'c23425f0-7bb3-4f54-88e8-7537cac19e58', 'x': [], 'y': []},
             {'name': '5% Greedy', 'type': 'scatter', 'uid': '299e3eb9-bc74-4b9c-8dc9-8e8acedab8d2', 'x': [], 'y': []}],
    'layout': {'title': {'text': 'Cumulative Regrets'}}
})

In [204]:
import plotly.offline as py
py.iplot(fig)

In [205]:
NB_STEPS = 10000


offset = len(fig.data[0].x)
print([net.label for lb, net in mushroom_nets.items()])

for _ in range(offset, offset+NB_STEPS):
    mushroom = _
    for j, (key, net) in enumerate(mushroom_nets.items()):
        net.update(mushroom)
    fig.update({net.label: net.cum_regrets for lb, net in mushroom_nets.items()})
    if _ % 10 == 0 :
        print('Step',_,'Regrets',[net.cum_regrets[-1] for lb, net in mushroom_nets.items()])

['BBB', 'Greedy', '1% Greedy', '5% Greedy']
Step 0 Regrets [5, 5, 5, 5]
Step 10 Regrets [20, 80, 25, 20]
Step 20 Regrets [45, 175, 50, 35]
Step 30 Regrets [70, 195, 70, 50]
Step 40 Regrets [85, 210, 80, 60]


KeyboardInterrupt: 